
    Copyright (C) 2021 Clement Galan (clement.galan@cri-paris.org)
                       Anton Crombach (anton.crombach@inria.fr)

**SYNOPSIS**

    generating_network_topologies.py [-h,--help] [-v,--verbose] [--version]

**DESCRIPTION**

    Provide functionality to compute all possible networks of a given
    number of genes (e.g. 3 or 4 genes). Output is written to file.
    
    Code is developed and written by Clement. A bit of beautification was 
    done by Anton. We verified that this script computes an isomorphic set of 
    graphs in comparison to the algorithm in the Jupyter notebook of Anton.

**EXAMPLES**

    python3 generating_network_topologies.py


HISTORICAL NOTES

#### THE SOFTWARE PACKAGE "Nauty"
----------------------------
We attempted to use a powerful package for graph analysis, though it was not
used in the end. See the following webpages for the relevant documentation.

https://computationalcombinatorics.wordpress.com/2012/08/05/tips-and-tricks-using-gtools/
http://users.cecs.anu.edu.au/~bdm/data/digraphs.html
https://www.geeksforgeeks.org/passing-function-as-an-argument-in-python/text=Because%20functions%20are%20objects%20we,a%20function%20as%20an%20argument

In [1]:
import sys
import os
import traceback
import argparse
import time

import numpy as np
import networkx as nx

In [2]:
##########################################
##### conversion graph <--> matrices #####
##########################################
def vector_to_matrix(vector_set):
    """For all network possibilities, build an adjacency matrix.

    Input: Array with as many rows as possible networks between N genes.
    Each row value corresponds to an interaction between two of the genes
    (including auto interaction).
    """
    n_genes = int(np.sqrt(len(vector_set[0])))
    vector_set = np.array(vector_set, dtype=int)
    matrix_set = np.reshape(vector_set, (len(vector_set), n_genes, n_genes))
    return list(matrix_set)

def convert_matrices_in_graphs(network_space):
    """For all network-adj matrices in the list, convert to a graph.
    
    Input: list of NxN arrays (being N the number of genes we consider).
    They could be obtained by inputting the N gene network space to 
    function vector_to_matrix"""
    return [nx.from_numpy_array(np.array(n), create_using=nx.DiGraph) 
            for n in network_space]

def convert_graphs_in_matrices(network_space):
    return [nx.to_numpy_array(n, dtype=int) 
            for n in network_space]

In [3]:
####################################################
##### creating the array of adjacency matrices #####
####################################################
def combine_networks(col, value, matrix):
    """
    Takes a row vector and modifies one of its coeficient to give it a new value.
    """
    new_branch = np.zeros(len(matrix), dtype=int)
    new_branch[col] = value
    return matrix + new_branch

def generate_space(network_space, col, size, list_param_values, n_genes):
    """
    Formalism used for the matrices:

    the coefficients of our interaction matrices follow the formalism (depending on the list_param_values):
    - param > max(list_param_values)/2 for activation interaction (different numbers correspond to different degrees of activation)
    - 0 < param <= max(list_param_values)/2 for inhibition interaction (different numbers correspond to different degrees of inhibition)
    - 0 for no interaction 
    
    each row represents a gene that is giving the signal
    each col represents a gene that is receiving the signal    
    
    INPUTS:
    - network_space is a list with 1D arrays that when reshaped, become square matrices (preferentially ONE ARRAY full of ZEROS)
    - col: column of the array from the list where to start the enumeration
    - size is the size of the array (n_genes*n_genes)

    WHAT THE FUNCTION DOES:
    It creates a list of 1D arrays, each corresponding to a combination of numbers in the list_param_values. 
    If the 1Darrays where converted to square matrices, the function is defined so that the last column is set to 0;
    this is becuse, for our problem, we need an external influence towards the genes. The way we model this is by 
    introducing an extra gene and allowing it only to have an effect on the other genes, but not being able to receive an
    effect of them or to have an auto interaction (or if it does have it, we dont care about it).
    
    This is done by going from one column (number in the 1D array) to the next one, each time changing a number and achieving a new configuration.
    Then it goes to the next row and for every combination that are already in 
    the network_space, it will give all the possible combination of parameters.
    The process is repeated for all rows
    
    In the end, this function will give all the possible combinations of parameter values that 
    exist for a given matrix size. return object is a list of row matrix.

    OUTPUT:
    List of 1D arrays of length n_genes*n_genes.

    IN THE FUTURE:
    From this set we want to remove all the isomorphisms and the weakly connected graphs to keep the unique graphs.
    """

    if col >= size :
        return network_space
    else:
        for n in network_space:
            if n[col] == 0:
                for i in range(len(list_param_values)):
                    network_space.append(combine_networks(col, list_param_values[i], n))
                    
        #N = n_genes-1
        #k = list(np.arange(N+1)*(N+1) + N)
        
        #if (col+1) in k:
         #   return generate_space(network_space, col+2, size, list_param_values, n_genes)
        #else:
        return generate_space(network_space, col+1, size, list_param_values, n_genes)

In [4]:
###########################################
#### removing non connected topologies ####
###########################################
def remove_weakly_connected(graph_set):
    """
    We remove the non weakly connected graphs, as we are not interested in these.
    """
    i = 0
    while i < len(graph_set):
        if nx.is_weakly_connected(graph_set[i]) == False:
            graph_set.remove(graph_set[i])
            i = i
        else: 
            i += 1
    return graph_set

In [5]:
########################################################
##### removing isomorphic topologies with networkx #####
########################################################

# Rationale:
#
# Then we compare all the graphs together and remove the isomorphisms
# Therefore we want to classify the matrices according to the number of interactions 
# and the type of interaction using the 'weight' function.
# With this classification we grouped all the matrices that can be isomorphs together.
# So we just check for isomorphisms among graphs of the same weight group.

def count_weight(matrix, list_param_values):
    """Calculate the weight (occurrence of a value) of a matrix."""
    weight= [0]*len(list_param_values)          # weight[0] = n° of 1 in the matrix
                                # weight[1] = n° of 2 in the matrix
    for j in range(len(list_param_values)):
        weight[j] = len([i for i in matrix.flat if i == list_param_values[j]])
    return weight


def sort_network_by_weight(count_weight, network_space, list_param_values):
    """Make a dictionnary with weight as a key and the corresponding list of networks as value."""
    sorted_networks = {}
    for n in network_space:
        sorted_networks.setdefault(tuple(count_weight(n, list_param_values)), []).append(n)
    return sorted_networks


def remove_isomorphism(network_space, em):
    """
    Intro list of equally weighted graphs, check if two graphs are isomorphic.
    If yes, one of the two is removed from the list.
    """
    j = 0
    while j < len(network_space):
        i = j+1
        while i < len(network_space):
            valid = nx.is_isomorphic(network_space[j], network_space[i], edge_match=em)
            if valid:
                network_space.pop(i)
                i = i
            else:
                i += 1
        j += 1
    return network_space

In [11]:
#######################################
##### Main function of the script #####
#######################################
def main():
    """
    Clement's script for generating all possible networks of a given size (nr of genes).
    
    The main difference with Anton's Jupyter notebook, is that here the entire space is generated 
    and then pruned. In Anton's version, networks are immediately pruned if possible. For bigger 
    spaces (n=4, n=5) that might make a difference in terms of memory consumption.
    
    We verified for network size n=3 that both Clement's and Anton's code generate an isomorphic 
    list of networks.
    """
    global args

    # To facilitate the computation we are using a line vector that can be converted into a line matrix
    list_param_values = np.arange(1,3,1)
    n_genes = 2
    #N = n_genes-1
    matrix_size = n_genes * n_genes
    initial_network = np.zeros(matrix_size, dtype=int) 
    initial_set = [initial_network]
    network_set = generate_space(initial_set, 0, matrix_size, list_param_values, n_genes)   
    print('net top size', len(network_set))
    # 0 mean we are starting to generate new coefficient values from the first coefficient to the last one

    network_set = vector_to_matrix(network_set)
    network_set = convert_matrices_in_graphs(network_set)
    network_set = remove_weakly_connected(network_set)
    network_set = convert_graphs_in_matrices(network_set)
    # Splitting the graphs in comparable groups
    network_set = sort_network_by_weight(count_weight, network_set, list_param_values)
    
    # Remove graphs and putting back groups together
    network_set_new = []
    em = nx.isomorphism.numerical_edge_match('weight', 'weight')

    for key in network_set:
        network_set[key] = convert_matrices_in_graphs(network_set[key])
        network_set[key] = remove_isomorphism(network_set[key], em)
        network_set_new.extend(network_set[key])

    network_set = network_set_new
    network_set = convert_graphs_in_matrices(network_set)

    # The set of topology that we have to consider is now created and will remain the same
    # So we save the data in a text file to make it more accessible for the other implementations
    network_set = np.array(network_set)
    network_set = np.reshape(network_set, (len(network_set), np.size(network_set[0])))

    # Remove rows with unwanted interactions of the 'external agent (N+1^th gene)' (those with
    # a different direction than two de other two genes (OUR GENES)
    # dele = np.array(range(1,(N+1+1)))*(N+1)-1
    # network_set = np.delete(network_set, dele, 1)
    # Save the data
    #np.savetxt("net_space_square_2.txt", network_set, '%d')      
    print('final', network_set)

In [12]:
main()

net top size 81
final [[0 1 0 0]
 [0 2 0 0]
 [1 1 0 0]
 [1 0 1 0]
 [0 1 1 0]
 [1 2 0 0]
 [2 1 0 0]
 [1 0 2 0]
 [2 0 1 0]
 [0 1 2 0]
 [2 2 0 0]
 [2 0 2 0]
 [0 2 2 0]
 [1 1 1 0]
 [1 1 0 1]
 [1 1 2 0]
 [1 2 1 0]
 [2 1 1 0]
 [1 1 0 2]
 [1 2 0 1]
 [2 1 0 1]
 [1 2 2 0]
 [2 1 2 0]
 [2 2 1 0]
 [1 2 0 2]
 [2 1 0 2]
 [2 2 0 1]
 [2 2 2 0]
 [2 2 0 2]
 [1 1 1 1]
 [1 1 1 2]
 [1 1 2 1]
 [1 1 2 2]
 [1 2 1 2]
 [1 2 2 1]
 [2 1 1 2]
 [1 2 2 2]
 [2 1 2 2]
 [2 2 2 2]]
